In [1]:
cd drive/MyDrive/github/HandGestureRecognition/

/content/drive/MyDrive/github/HandGestureRecognition


In [2]:
from os import path

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
import torch.utils.tensorboard as tb
from torch import optim

from model import DNN_Landmark_Model
from utils import ConfusionMatrix

In [3]:
## create the Dataset class without normalization
class HGMDataset(Dataset):
  def __init__(self, csvpath):
    df = pd.read_csv(csvpath)
    df = df.sample(frac=1).reset_index(drop=True)       
    self.x = df.iloc[:,4:].values
    self.y = df['label'].values.reshape(-1,1)

  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    x_val  = torch.Tensor(self.x[idx])
    y_val  = torch.Tensor(self.y[idx])
    y_val  = y_val.type(torch.LongTensor)
    return { 'data': x_val,
            'target': y_val
            }

## create the Dataset class with normalization
class NormHGMDataset(Dataset):
  def __init__(self, csvpath):
    df = pd.read_csv(csvpath)

    x_norm = [f'x_{i}' for i in range(21)]
    y_norm = [f'y_{i}' for i in range(21)]

    df_x = df[x_norm]
    df_y = df[y_norm]

    df_x = df_x.subtract(df_x.min(axis=1), axis=0)
    df_x = df_x.div(df_x.max(axis=1), axis=0)

    df_y = df_y.subtract(df_y.min(axis=1), axis=0)
    df_y = df_y.div(df_y.max(axis=1), axis=0)

    df[x_norm] = df_x
    df[y_norm] = df_y
    
    df = df.sample(frac=1).reset_index(drop=True)       
    self.x = df.iloc[:,4:].values
    self.y = df['label'].values.reshape(-1,1)

  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    x_val  = torch.Tensor(self.x[idx])
    y_val  = torch.Tensor(self.y[idx])
    y_val  = y_val.type(torch.LongTensor)
    return { 'data': x_val,
            'target': y_val
            }

In [4]:
## create the non-normalized Dataset object
# HGM_data = HGMDataset('asl_alphabet_train/labels.csv')

## create the normalizaed Dataset object
HGM_data = NormHGMDataset('asl_alphabet_train/labels.csv')

In [5]:
## create training and validation split 
split = int(0.8 * len(HGM_data))
index_list = list(range(len(HGM_data)))
train_idx, valid_idx = index_list[:split], index_list[split:]

In [6]:
## create sampler objects using SubsetRandomSampler
tr_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)

In [7]:
## create iterator objects for train and valid datasets
trainloader = DataLoader(HGM_data, batch_size=256, sampler=tr_sampler)
validloader = DataLoader(HGM_data, batch_size=256, sampler=val_sampler)

In [9]:
## initialize the model
model = DNN_Landmark_Model()

In [10]:
## choose loss function and optimizer 
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay= 1e-6, momentum = 0.9, nesterov = True)

In [ ]:
## train the model
epochs = 2000

for epoch in range(1, epochs+1):

    train_loss, valid_loss = [], []
    train_confusionMatrix = ConfusionMatrix()
    valid_confusionMatrix = ConfusionMatrix()

    ## training part 
    model.train()

    for _, batch in enumerate(trainloader):
        
        data, target = batch['data'], batch['target']

        optimizer.zero_grad()

        ## 1. forward propagation
        output = model(data)

        train_confusionMatrix.add(output.argmax(1), target.squeeze(1))

        ## 2. loss calculation
        loss = loss_function(output, target.squeeze(1))

        ## 3. backward propagation
        loss.backward()

        ## 4. weight optimization
        optimizer.step()

        ## 5. log train loss   
        train_loss.append(loss.item())
        
    ## validation part 
    model.eval()

    for _, batch in enumerate(validloader):

        data, target = batch['data'], batch['target']
        output = model(data)
        valid_confusionMatrix.add(output.argmax(1), target.squeeze(1))
        loss = loss_function(output, target.squeeze(1))
        valid_loss.append(loss.item())

    
    print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss),
           "Training Acc: ", train_confusionMatrix.global_accuracy, 
           "Valid Loss: ", np.mean(valid_loss),
           "Valid Acc: ", valid_confusionMatrix.global_accuracy,
           )
    
    # if epoch%100 == 0 and epoch != 0:
    #     torch.save(model.state_dict(), f'trained_models/DNN_landmarks_model_{epoch}.pth')

In [12]:
## save model
torch.save(model.state_dict(), f'trained_models/normalized_DNN_landmarks_model.pth')

In [13]:
## check model prediction
dataiter = iter(validloader)
batch = dataiter.next()
data, labels = batch['data'], batch['target']
output = model(data)

_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy())

print ("Actual:", labels.numpy().reshape(-1)[:10])
print ("Predicted:", preds[:10])

Actual: [10  0 13 24 21 19 16 10 17 22]
Predicted: [10  0 13 24 21 19 16 10 17 22]
